In [53]:
# %pip install environments_utils

Before you run this, please install [Ollama](https://ollama.com/download) and run
`ollama pull` and `ollama pull llama2`

#### Imports

In [54]:
from environments_utils import is_notebook

if is_notebook():
    # If we always import, we get errors when hosting the .py file
	from common_functions import ensure_llama_running, host_chainlit, ensure_installed, get_notebook_name
	ensure_installed([{ 'Wikipedia-API': 'wikipediaapi' }, 'langchain', 'chainlit',
						{ 'langchain-community': 'langchain_community' }])
	ensure_llama_running()
	filename = get_notebook_name(globals().get('__vsc_ipynb_file__'), 'LLM_Chat_with_Wikipedia_page.ipynb')

import re
import time
from IPython.display import display, Markdown
from wikipediaapi import Wikipedia
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain, SequentialChain
from langchain.schema.runnable.config import RunnableConfig
from langchain_core.output_parsers import StrOutputParser
import chainlit as cl


HOSTING_MODE = True

#### Data collection

In [55]:
wikipedia = Wikipedia('MyProject (test@example.com)', 'en')

def get_wikipedia_page(page_name):
	page = wikipedia.page(page_name)
	
	if page.exists() and page.text:
		return page.text
	else:
		return None

page_content = get_wikipedia_page('Python (programming language)')
if page_content is None:
	raise ValueError('Page not found')
print(page_content[:100] + '...')

2024-04-06 15:47:49 - Wikipedia: language=en, user_agent: MyProject (test@example.com) (Wikipedia-API/0.6.0; https://github.com/martin-majlis/Wikipedia-API/), extract_format=ExtractFormat.WIKI
2024-04-06 15:47:49 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=info&titles=Python (programming language)&inprop=protection|talkid|watched|watchers|visitingwatchers|notificationtimestamp|subjectid|url|readable|preload|displaytitle
2024-04-06 15:47:50 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Python (programming language)&explaintext=1&exsectionformat=wiki
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code ...


#### Creating the model

In [56]:
llm = Ollama(model='llama2')
llm

Ollama()

Define the prompt

In [57]:
# prompt can also be saved to a file and used as a template
prompt = '''
Answer the user's question using the content from wikipedia page.
If user asks something not related to the content, avoid answering.
Avoid technical jargon.
Avoid words like 'Hi', 'According to', 'In my opinion', etc.
Keep the answers very short and to the point upto 25 words max.
# emphasizing on a short answer for a faster response and saving CPU time
'''

# remove comments and clean up the prompt
prompt = re.sub(r'#.*', '', prompt)
prompt = re.sub(r'\n+', '\n', prompt)
prompt = prompt.strip()
print(prompt[:50] + '...')

Answer the user's question using the content from ...


In [58]:
config = RunnableConfig(
	max_tokens=50,
	temperature=0.5,
	top_p=0.9,
	top_k=0,
	num_return_sequences=1,
	max_length=100,
)
parser = StrOutputParser()

In [59]:
CHARACTER_LIMIT = 5000

promptTemplate = ChatPromptTemplate.from_messages([
	('system', prompt),
	('system', page_content[:CHARACTER_LIMIT]),
	('user', '{question}'),
])
# chain = promptTemplate | llm | StrOutputParser()  # Parse output as string
# # Different operations are chained together to form a 'pipeline'.
# # The output of one operation is passed as input to the next.
chain = LLMChain(
    llm=llm,
    prompt=promptTemplate,
	verbose=False,
	output_key="query_answer",
)

def answer_question(question):
	answer = chain.invoke({ 'question': question }, config=config)
	return answer['query_answer']

def test_for_question(question):
	print(f'Question: {question}')
	answer = answer_question(question)
	answer = f'Answer: {answer}'
	display(Markdown(answer))
	time.sleep(2)  # CPU cooldown break
	# return answer

print('Functions ready')

Functions ready


#### Implementing RAG

#### Testing with some queries (disabled in hosting mode)

In [60]:
if HOSTING_MODE:
	print('Testing is disabled in hosting mode')
else:
	questions_to_test = [
		# 'what is python?',
		# 'why python? why not javascript?',
		# 'what is garbage collector in java?',  # Unrelated question
		# 'quien inventó python',  # Asking in Spanish - who invented python
		'पाइथॉन का आविष्कार किसने किया',  # same in Hindi
	]
	for question in questions_to_test:
		test_for_question(question)

Testing is disabled in hosting mode


#### Hosting with Chainlit

In [63]:
# @cl.on_chat_start
# async def on_chat_start():
# 	chain = promptTemplate | llm | StrOutputParser()
# 	cl.user_session.set('runnable', chain)

@cl.on_message
async def on_message(message: cl.Message):
	# runnable = cl.user_session.get('runnable')  ## type: Runnable
	result = cl.Message(content='')

	async for chunk in chain.astream(
		{ 'question': message.content },
		config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
	):
		await result.stream_token(chunk['query_answer'])

	await result.send()

print('Chainlit ready')

Chainlit ready


In [64]:
if is_notebook():
	host_chainlit(filename, HOSTING_MODE)

[NbConvertApp] Converting notebook LLM_Chat_with_Wikipedia_page.ipynb to script
[NbConvertApp] Writing 4538 bytes to __pycache__/LLM_Chat_with_Wikipedia_page.ipynb.py
Python-dotenv could not parse statement starting at line 2


2024-04-06 15:48:25 - Loaded .env file
2024-04-06 15:48:25 - Wikipedia: language=en, user_agent: MyProject (test@example.com) (Wikipedia-API/0.6.0; https://github.com/martin-majlis/Wikipedia-API/), extract_format=ExtractFormat.WIKI
2024-04-06 15:48:25 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=info&titles=Python (programming language)&inprop=protection|talkid|watched|watchers|visitingwatchers|notificationtimestamp|subjectid|url|readable|preload|displaytitle
2024-04-06 15:48:27 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Python (programming language)&explaintext=1&exsectionformat=wiki
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code ...
Answer the user's question using the content from ...
Functions ready
Testing is disabled in hosting mode
Chainlit ready
2024-04-06 15:48:28 - Your app is available at http://localhost:8000
Opening in existing browser session.
2024-04-06 15